<a href="https://colab.research.google.com/github/Phuongthao2k/CS114.K21.KHTN/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 18521422 - Trần Thị Phương Thảo
 18521348 - Lưu Hoàng Sơn

## Sarcasm Indetifier
https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection  
- Input: 1 headline
- Ouput: input headline is Sarcastic or not

### Data collection
Train Data: From Kaggle  
Test Data: From huffpost & the onion


In [ ]:
!wget https://raw.githubusercontent.com/sonlhcsuit/CS114.K21.KHTN/master/Sarcasm_Headlines_Dataset.json

--2020-06-25 15:41:53--  https://raw.githubusercontent.com/sonlhcsuit/CS114.K21.KHTN/master/Sarcasm_Headlines_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5616830 (5.4M) [text/plain]
Saving to: ‘Sarcasm_Headlines_Dataset.json’

Sarcasm_Headlines_D 100%[===================>]   5.36M  13.2MB/s    in 0.4s    

2020-06-25 15:41:54 (13.2 MB/s) - ‘Sarcasm_Headlines_Dataset.json’ saved [5616830/5616830]



In [ ]:
import pandas as pd
kaggleData = pd.read_json('/content/Sarcasm_Headlines_Dataset.json',lines=True)
kaggleData.sample(10)

,article_link,headline,is_sarcastic
12957,https://local.theonion.com/chipotle-mayo-doing...,chipotle mayo doing all the heavy lifting in s...,1
21951,https://www.theonion.com/ponds-institute-tops-...,ponds institute tops 1997 cosmopolitan college...,1
17527,https://www.huffingtonpost.com/entry/joey-ches...,watch: what it feels like after you wolf down ...,0
15640,https://www.theonion.com/report-96-of-nations-...,report: 96% of nation's smut consumed by filth...,1
9331,https://www.huffingtonpost.com/entry/cash-back...,cash back incentives: a winning strategy for ...,0
982,https://www.theonion.com/newborn-prince-of-cam...,newborn prince of cambridge begins consolidati...,1
12256,https://entertainment.theonion.com/mpaa-unveil...,mpaa unveils rating system based on old testament,1
23831,https://www.huffingtonpost.com/entry/trainspot...,revisit your favorite junkie pals in the 'trai...,0
26395,https://www.huffingtonpost.com/entry/margaret-...,margaret atwood's advice for young feminists: ...,0
18810,https://www.huffingtonpost.com/entry/five-minu...,5-minute hairstyles -- for real!,0


In [ ]:
kaggleData.headline[3537]

"fans erupt over fate of elias koteas' olinsky on 'chicago p.d.'"

In [ ]:
data = kaggleData.drop(columns='article_link')
data.head(10)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
5,advancing the world's women,0
6,the fascinating case for eating lab-grown meat,0
7,"this ceo will send your kids to school, if you...",0
8,top snake handler leaves sinking huckabee camp...,1
9,friday's morning email: inside trump's presser...,0


#PreProcessData

In [ ]:
import re 
def pre_process_data(headline):
  test = headline.lower()
  spaceRemove = re.compile("[\s]")
  BoDauCau = re.compile('[\.\!\,\:\'\"\(\)\@\#\$\^\&\*\’\‘]')
  abc = re.sub(BoDauCau,'',test)
  abc = re.sub(spaceRemove,"|",abc)
  abc = re.sub('((\|[a-z]\|)|(\|an\|)|(\|the\|)|(\|this\|)|(\|that\|)|(\|these\|)|(\|those\|))','|',abc)
  abc = re.sub('\|+',' ',abc)
  return abc.strip()
test = "fans erupt over fate of elias koteas' olinsky on 'chicago p.d.'"
pre_process_data(test)

'fans erupt over fate of elias koteas olinsky on chicago pd'

In [ ]:
data.headline = data.headline.apply(pre_process_data)
data.sample(10)

,headline,is_sarcastic
14494,is what it was like to go to airport before 9/11,0
4080,what should we believe marco rubio or math?,0
22661,impressive new hire figures out bare minimum o...,1
14268,las vegas reviewjournal staff balks at limits ...,0
3833,bold intern giving parents tour of office,1
10350,concert spent const tly verifying presence of ...,1
10740,medicaid exp sion tied to employment among peo...,0
24689,psychic fall cometh,0
6513,sh non tweed named head of us task force on nu...,1
21308,streetsmart teen dies in library,1


#Train 

In [ ]:
from sklearn.model_selection import train_test_split
X = data.headline
Y = data.is_sarcastic
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

trainVector = CountVectorizer(analyzer='word',ngram_range=(1,1), max_features=10000) #make a vectorizer for train dataset
X_train = trainVector.fit_transform(X_train) #vectorize train set to get features 

testVector = CountVectorizer(vocabulary=trainVector.vocabulary_,ngram_range=(1,1)) #make a vectorize for test set, using train set vocabulary_
X_test = testVector.fit_transform(X_test) #vectorize test set to get features

## Select Model

##SVC 

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
SVClassifiers = SVC()
SVClassifiers.fit(X_train, Y_train, )
SVC_predict = SVClassifiers.predict(X_test)
SCV_score = accuracy_score(Y_test, SVC_predict)
SCV_score

0.819543242231374

### Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

LinearSVClassifier = LinearSVC()
LinearSVClassifier.fit(X_train, Y_train)
Linear_predict = LinearSVClassifier.predict(X_test)
Linearscore = accuracy_score(Y_test, Linear_predict)
Linearscore 

0.7962061649819044

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFClassifier = RandomForestClassifier()
RFClassifier.fit(X_train, Y_train)
RFClassifier_predict = RFClassifier.predict(X_test)
RFClassifier_score = accuracy_score(Y_test, RFClassifier_predict)
RFClassifier_score
print(RFClassifier.score(X_test, Y_test))

0.7771121926868838


## TEST ON DATA 

In [ ]:
!wget https://raw.githubusercontent.com/sonlhcsuit/CS114.K21.KHTN/master/CRAWLED_DATA.json

--2020-06-25 16:52:59--  https://raw.githubusercontent.com/sonlhcsuit/CS114.K21.KHTN/master/CRAWLED_DATA.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229984 (225K) [text/plain]
Saving to: ‘CRAWLED_DATA.json’

CRAWLED_DATA.json   100%[===================>] 224.59K  --.-KB/s    in 0.04s   

2020-06-25 16:52:59 (5.28 MB/s) - ‘CRAWLED_DATA.json’ saved [229984/229984]



In [ ]:
CRAWLED = pd.read_json('/content/CRAWLED_DATA.json',lines=True)
CRAWLED.head(10)

,headline,is_sarcastic
0,ExxonMobil Simplifies Oil Extraction By Cuttin...,1
1,Quaker Oats Replaces Historically Racist Aunt ...,1
2,Showrunner Worried This A Bad Time For New Ser...,1
3,Anna Kendrick’s ‘Love Life’ Pulled From HBO Ma...,1
4,South Secedes From NASCAR Following Confederat...,1
5,No Way In Hell Man Going To Start Examining Re...,1
6,White Ally Willing To Do Whatever It Takes To ...,1
7,Woman On Sidewalk Can’t Even Summon Kernel Of ...,1
8,37-Year-Old Worried He Might Have Missed Boat ...,1
9,Biden Flattered His 1994 Crime Bill Suddenly S...,1


In [ ]:
CRAWLED.headline = CRAWLED.headline.apply(pre_process_data)
CRAWLED

,headline,is_sarcastic
0,exxonmobil simplifies oil extraction by cuttin...,1
1,quaker oats replaces historically racist aunt ...,1
2,showrunner worried a bad time for new series a...,1
3,anna kendricks love life pulled from hbo max u...,1
4,south secedes from nascar following confederat...,1
...,...,...
1995,susan rice explains exactly why trumps white h...,0
1996,scathing supercut shows how quickly trump flip...,0
1997,trumps racist toddler video removed by faceboo...,0
1998,50 years after mlk pastors lead new poor peopl...,0


In [ ]:
X_craw = CRAWLED.headline
Y_craw = CRAWLED.is_sarcastic

X_craw_test = testVector.fit_transform(X_craw)

In [ ]:
from sklearn.metrics import classification_report
for cls in [SVClassifiers,LinearSVClassifier,RFClassifier]:
    ypred = cls.predict(X_craw_test)
    yscore = cls.score(X_craw_test, Y_craw)
    print(classification_report(Y_craw, ypred))
    print(cls.__class__.__name__, yscore)

              precision    recall  f1-score   support

           0       0.76      0.86      0.81       985
           1       0.85      0.73      0.78      1015

    accuracy                           0.80      2000
   macro avg       0.80      0.80      0.80      2000
weighted avg       0.80      0.80      0.80      2000

SVC 0.796
              precision    recall  f1-score   support

           0       0.74      0.78      0.76       985
           1       0.77      0.73      0.75      1015

    accuracy                           0.76      2000
   macro avg       0.76      0.76      0.75      2000
weighted avg       0.76      0.76      0.75      2000

LinearSVC 0.755
              precision    recall  f1-score   support

           0       0.70      0.84      0.76       985
           1       0.81      0.65      0.72      1015

    accuracy                           0.74      2000
   macro avg       0.75      0.75      0.74      2000
weighted avg       0.75      0.74      0.74     

DATA CRAWLED FROM : 
https://github.com/sonlhcsuit/CS114.K21.KHTN/edit/master/CRAWDATA.py